In [2]:
import torch
# import libraries
from managers.shiyongren import Manager
from models.shiyongren import D2NNmodel
from helper import *

In [ ]:
n_layers = 4
n_size_per_layer = 400
distance_list = torch.tensor([29.5, 29.5, 29.5, 71]) * 1e-3
pixel_pitch = 8e-6
lambd_list = torch.tensor([491e-9, 660e-9])
n_padding = 200
lr = 1e-3
batch_size = 12
whether_load_model = False
load_model_name = 'saved_model/shiyongren.pth'
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
load_dic = [
    {'path':'input_field_lamda_12.mat', 'vari_name':'U_load'},
    {'path':'output_filed_lamda_12.mat', 'vari_name':'Y_load'}
]
dataloaders = shiyong_loader(load_dic, batch_size)

dict_keys(['__header__', '__version__', '__globals__', 'U_load'])


In [ ]:
model = D2NNmodel(pixel_pitch, distance_list, n_padding, device, lambd_list, n_size_per_layer, 0)
loss_fn = mse_loss()
optimizer = optim.Adam(model.parameters(), lr=lr)
manager = Manager(model, loss_fn, optimizer)
manager.set_loaders(dataloaders, dataloaders)
# tracking, fig = manager.lr_range_test(train_dataloader, end_lr=1e-1, num_iter=100)
if whether_load_model:
    manager.load_checkpoint(load_model_name, None)

In [ ]:
# manager.train(6)

In [ ]:
fig = manager.plot_losses()

In [ ]:
# manager.save_checkpoint('saved_model/07_22_2024.pth')

In [ ]:
layers_name = ['modulation_1', 'modulation_2', 'modulation_3']
fig_mask_height = manager.visualize_height_mask(layers_name, hmin, hmax, bit_depth)

In [ ]:
# A_hat = manager.get_transform_tensor(A)
fig_1, mean_error = manager.figure_1(A)
print(mean_error)

In [ ]:
fig_2 = manager.figure_2()

In [ ]:
# To be continued ...

In [ ]:
print()